In [2]:
# Install BigQuery client library (run this cell once)
%pip install google-cloud-bigquery --quiet
%pip install bigquery-magics --quiet
%pip install db-dtypes --quiet
# Import libraries
from google.cloud import bigquery
import pandas as pd
%load_ext bigquery_magics
import db_dtypes
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "salesforce-465614-2cf9e37da64b.json"
from google.cloud import bigquery
# Initialize BigQuery client
client = bigquery.Client()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [5]:
%%bqsql
select * from salesforce-465614.cust_analytics.fact_transactions
limit 2

Query is running:   0%|          |

Downloading:   0%|          |

,transaction_id,customer_id,product_id,Store_id,Purchase_Date,Unit_Price,quantity,Total_Price,Payment_Method,Order_Status,start_date,end_date,is_current
0,T234143,C1119,P102,STORE_32,2024-08-26,504.81,1,504.81,Cash,Completed,2025-07-17,NaT,True
1,T526357,C1468,P102,STORE_44,2024-01-24,504.81,1,504.81,Bank Transfer,Cancelled,2025-07-17,NaT,True


<h2>Reporting Layer</h2>

In [5]:
%%bqsql
drop table if exists salesforce-465614.cust_analytics.report_transactions;

Query is running:   0%|          |

""


In [6]:
%%bqsql
CREATE TABLE salesforce-465614.cust_analytics.report_transactions
PARTITION BY Purchase_Date
CLUSTER BY customer_id, product_id, Store_id
as
select 
transaction_id,fact.customer_id,fact.product_id,fact.Store_id,order_status,
Purchase_Date,category,sub_category,Unit_Price,quantity,Total_Price,payment_method,store_type as channel,loyalty_program,
age,gender,city from 
(select * from salesforce-465614.cust_analytics.fact_transactions where is_current=True) fact
left join 
(select * from salesforce-465614.cust_analytics.dim_customers where is_current=True)  cust
on fact.customer_id=cust.customer_id
left join 
(select * from salesforce-465614.cust_analytics.dim_products where is_current=True) product 
on fact.product_id=product.product_id
left join 
(select * from salesforce-465614.cust_analytics.dim_stores where is_current=True) stores
on fact.Store_id=stores.Store_id

Query is running:   0%|          |

""


In [ ]:
## Optional columns to consider for reporting 
## first_name,last_name,gender,age,signup_date,loyalty_program,email,city,state,country,
##  store_type,store_location,

In [6]:
%%bqsql
select * from salesforce-465614.cust_analytics.dim_stores where store_location='STORE_32'

Query is running:   0%|          |

Downloading: |          |

,store_id,store_type,store_location,region,start_date,end_date,is_current
